## Libraries Setup

In [4]:
!pip install -r requirements.txt -q

In [5]:
# Datahandling
import requests
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import numpy as np
import pandas as pd
import seaborn as sns

# Tweet Themes
from sentence_transformers import SentenceTransformer
import umap
import hdbscan
import matplotlib.pyplot as plt
import plotly.express as px

# Network analysis
from typing import List, Optional
import json
import matplotlib.patches as mpatches
import networkx as nx
from community import community_louvain
import plotly.graph_objects as go
import random
from collections import defaultdict
import nbformat

# Model prediction
from setfit import SetFitModel, SetFitTrainer
from sklearn.metrics import classification_report
import tiktoken

# Gradio deployment
import gradio as gr


/opt/anaconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Data import

In [9]:
# data sampling? 20000
data = pd.read_csv('predictions_output.csv')
# data = data.sample(n=2000, random_state=42)

In [10]:
# Split the data based on the `label` column
data_label_0 = data[data['Label'] == 0]  # Subset where label=0
data_label_1 = data[data['Label'] == 1]  # Subset where label=1

# Optionally, check the sizes of the splits
print(f"Data with label=0: {len(data_label_0)} rows")
print(f"Data with label=1: {len(data_label_1)} rows")

Data with label=0: 138 rows
Data with label=1: 49 rows


In [11]:
# LLM Libs & Setup
from openai import OpenAI
import json
from pydantic import BaseModel, Field
from typing import List, Optional
import textwrap

## LLM Setup

In [ ]:
# Set your Together API Key directly
TOGETHER_API_KEY = ""

# Check if the API key is set
if not TOGETHER_API_KEY:
    raise ValueError("The Together API Key must be provided.")

# Initialize the Together client
client = OpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=TOGETHER_API_KEY
)

# Define the model
model = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"

## System_Prompt + JSON_schema Setup

In [13]:
SYSTEM_PROMPT = f"""
You are an advanced theme extraction and analysis assistant specializing in comprehensive thematic breakdown of a collection of tweets.


Extraction Objectives:
1. Identify and Extract Overall Themes
2. Determine Theme Centrality and Significance
3. Uncover Sub-Themes and Causal Relationships
4. Provide Nuanced Thematic Insights


Output Structure:
{{
    "overall_themes": [
        {{
            "name": "Primary Theme Name",
            "confidence": "Confidence level 0-100",
            "centrality": {{
                "weight": "Theme heaviness/importance (0-100)",
                "calculation_factors": [
                    "Tweet volume",
                    "Sentiment intensity",
                    "Keyword repetition",
                    "Cross-tweet relevance"
                ]
            }},
            "sub_themes": [
                {{
                    "name": "Specific Sub-Theme",
                    "causations": [
                        "Underlying factors driving this sub-theme",
                        "Contextual triggers"
                    ],
                    "keywords": ["Supporting keywords"],
                    "sentiment": "positive/negative/neutral",
                    "confidence": "Sub-theme confidence level 0-100"
                }}
            ],
            "keywords": ["Primary theme defining keywords"],
            "sentiment_distribution": {{
                "positive_percentage": 0,
                "negative_percentage": 0,
                "neutral_percentage": 0
            }}
        }}
    ],
    "metadata": {{
        "total_tweets_analyzed": 0,
        "unique_themes_count": 0,
        "dominant_sentiment": "",
        "language_distribution": {{}}
    }}
}}


Analytical Guidelines:


1. Theme Identification:
- Limit overall themes to a maximum of 10
- Ensure themes are distinct and non-overlapping
- Capture the essence of the tweet collection


2. Centrality and Significance:
- Calculate theme weight using:
  * Number of tweets referencing the theme
  * Sentiment intensity
  * Keyword frequency
  * Cross-tweet relevance
  * Emotional depth


3. Sub-Theme and Causation Analysis:
- Identify specific sub-themes within each overall theme
- Uncover potential causative factors
- Provide context for theme emergence


4. Sentiment and Nuance:
- Analyze emotional tone at theme and sub-theme levels
- Provide percentage-based sentiment distribution
- Highlight emotional complexity


5. Metadata Insights:
- Track total tweets analyzed
- Count unique themes
- Capture language diversity
- Identify dominant sentiment


Principles:
- Objectivity: Base analysis solely on tweet content
- Precision: Provide granular, specific insights
- Consistency: Apply uniform analytical framework
- Depth: Go beyond surface-level theme extraction


Handling Complexity:
- For ambiguous themes, use lower confidence scores
- Highlight keywords that support theme identification
- Provide nuanced explanations for theme relationships


Ethical Considerations:
- Maintain data integrity
- Avoid external assumptions
- Respect contextual limitations of the dataset


Output Requirements:
- Strictly adhere to the specified JSON schema
- Ensure clear, structured, and insightful presentation
"""

In [14]:
json_schema = f"""{{
    "type": "object",
    "properties": {{
        "overall_themes": {{
            "type": "array",
            "maxItems": 10,
            "items": {{
                "type": "object",
                "properties": {{
                    "name": {{
                        "type": "string",
                        "description": "Primary theme name"
                    }},
                    "confidence": {{
                        "type": "number",
                        "minimum": 0,
                        "maximum": 100,
                        "description": "Confidence level of theme identification"
                    }},
                    "centrality": {{
                        "type": "object",
                        "properties": {{
                            "weight": {{
                                "type": "number",
                                "minimum": 0,
                                "maximum": 100,
                                "description": "Theme heaviness/importance"
                            }},
                            "calculation_factors": {{
                                "type": "array",
                                "items": {{
                                    "type": "string"
                                }},
                                "description": "Factors used to calculate theme centrality"
                            }}
                        }},
                        "required": ["weight"]
                    }},
                    "sub_themes": {{
                        "type": "array",
                        "items": {{
                            "type": "object",
                            "properties": {{
                                "name": {{
                                    "type": "string",
                                    "description": "Specific sub-theme name"
                                }},
                                "causations": {{
                                    "type": "array",
                                    "items": {{
                                        "type": "string"
                                    }},
                                    "description": "Underlying factors driving the sub-theme"
                                }},
                                "keywords": {{
                                    "type": "array",
                                    "items": {{
                                        "type": "string"
                                    }},
                                    "description": "Supporting keywords for the sub-theme"
                                }},
                                "sentiment": {{
                                    "type": "string",
                                    "enum": ["positive", "negative", "neutral"],
                                    "description": "Emotional tone of the sub-theme"
                                }},
                                "confidence": {{
                                    "type": "number",
                                    "minimum": 0,
                                    "maximum": 100,
                                    "description": "Confidence level of sub-theme identification"
                                }}
                            }},
                            "required": ["name"]
                        }},
                        "description": "Sub-themes within the primary theme"
                    }},
                    "keywords": {{
                        "type": "array",
                        "items": {{
                            "type": "string"
                        }},
                        "description": "Key terms supporting the primary theme"
                    }},
                    "sentiment_distribution": {{
                        "type": "object",
                        "properties": {{
                            "positive_percentage": {{
                                "type": "number",
                                "minimum": 0,
                                "maximum": 100
                            }},
                            "negative_percentage": {{
                                "type": "number",
                                "minimum": 0,
                                "maximum": 100
                            }},
                            "neutral_percentage": {{
                                "type": "number",
                                "minimum": 0,
                                "maximum": 100
                            }}
                        }}
                    }}
                }},
                "required": ["name", "confidence"]
            }}
        }},
        "metadata": {{
            "type": "object",
            "properties": {{
                "total_tweets_analyzed": {{
                    "type": "integer",
                    "minimum": 0
                }},
                "unique_themes_count": {{
                    "type": "integer",
                    "minimum": 0
                }},
                "dominant_sentiment": {{
                    "type": "string",
                    "enum": ["positive", "negative", "neutral"]
                }},
                "language_distribution": {{
                    "type": "object",
                    "additionalProperties": {{
                        "type": "integer"
                    }}
                }}
            }}
        }}
    }},
    "required": ["overall_themes"]
}}"""

In [15]:
from pydantic import BaseModel, Field
from typing import List, Optional, Dict


class Centrality(BaseModel):
    weight: float = Field(
        description="Theme heaviness/importance",
        ge=0,
        le=100
    )
    calculation_factors: Optional[List[str]] = Field(
        default=None,
        description="Factors used to calculate theme centrality"
    )


class SubTheme(BaseModel):
    name: str = Field(description="Specific sub-theme name")
    causations: Optional[List[str]] = Field(
        default=None,
        description="Underlying factors driving the sub-theme"
    )
    keywords: Optional[List[str]] = Field(
        default=None,
        description="Supporting keywords for the sub-theme"
    )
    sentiment: Optional[str] = Field(
        default=None,
        description="Emotional tone of the sub-theme",
        pattern="^(positive|negative|neutral)$"
    )
    confidence: Optional[float] = Field(
        default=None,
        description="Confidence level of sub-theme identification",
        ge=0,
        le=100
    )


class SentimentDistribution(BaseModel):
    positive_percentage: float = Field(ge=0, le=100)
    negative_percentage: float = Field(ge=0, le=100)
    neutral_percentage: float = Field(ge=0, le=100)


class OverallTheme(BaseModel):
    name: str = Field(description="Primary theme name")
    confidence: float = Field(
        description="Confidence level of theme identification",
        ge=0,
        le=100
    )
    centrality: Optional[Centrality] = Field(
        default=None,
        description="Theme centrality and importance"
    )
    sub_themes: Optional[List[SubTheme]] = Field(
        default=None,
        description="Sub-themes within the primary theme"
    )
    keywords: Optional[List[str]] = Field(
        default=None,
        description="Key terms supporting the primary theme"
    )
    sentiment_distribution: Optional[SentimentDistribution] = Field(
        default=None,
        description="Sentiment distribution for the theme"
    )


class Metadata(BaseModel):
    total_tweets_analyzed: Optional[int] = Field(
        default=None,
        description="Total number of tweets analyzed",
        ge=0
    )
    unique_themes_count: Optional[int] = Field(
        default=None,
        description="Number of unique themes identified",
        ge=0
    )
    dominant_sentiment: Optional[str] = Field(
        default=None,
        description="Overall dominant sentiment",
        pattern="^(positive|negative|neutral)$"
    )
    language_distribution: Optional[Dict[str, int]] = Field(
        default=None,
        description="Distribution of languages in the tweets"
    )


class TweetAnalysis(BaseModel):
    overall_themes: List[OverallTheme] = Field(
        description="List of overall themes",
        max_items=10
    )
    metadata: Optional[Metadata] = Field(
        default=None,
        description="Metadata about the tweet collection"
    )


# Dynamically generate the schema
json_schema = TweetAnalysis.model_json_schema()

/opt/anaconda3/lib/python3.12/site-packages/pydantic/fields.py:774: PydanticDeprecatedSince20: `max_items` is deprecated and will be removed, use `max_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warn('`max_items` is deprecated and will be removed, use `max_length` instead', DeprecationWarning)


## LLM-runtime

In [16]:
import tiktoken
import pandas as pd
import json
import time
from concurrent.futures import ThreadPoolExecutor
from queue import Queue
import threading




def rate_limited_executor(max_calls_per_minute=100):
    """Rate-limited executor for API calls."""
    call_queue = Queue()




    def worker():
        while True:
            func, args, kwargs, result_queue = call_queue.get()
            try:
                result_queue.put(func(*args, **kwargs))
            except Exception as e:
                result_queue.put(e)
            call_queue.task_done()
            time.sleep(60 / max_calls_per_minute)




    for _ in range(max_calls_per_minute):
        threading.Thread(target=worker, daemon=True).start()




    def submit(func, *args, **kwargs):
        result_queue = Queue()
        call_queue.put((func, args, kwargs, result_queue))
        result = result_queue.get()
        if isinstance(result, Exception):
            raise result
        return result




    return submit




def process_batch(tweets, executor, model, schema, batch_size):
    """Process one batch of tweets."""
    messages = [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': json.dumps(tweets)}
    ]
    try:
        response = executor(
            client.chat.completions.create,
            model=model,
            messages=messages,
            temperature=0.7,
            response_format={"type": "json_object", "schema": schema},
            max_tokens=8192
        )
        return {
            'batch_size': len(tweets),
            'response': response.choices[0].message.content
        }
    except Exception as e:
        return {
            'batch_size': len(tweets),
            'error': str(e)
        }




def parallel_process_tweets(data, batch_size, max_workers, model, schema):
    """Parallel processing of tweets with rate-limiting."""
    executor = rate_limited_executor(max_calls_per_minute=60)
    results = []




    def process_and_collect(batch):
        tweets = batch['Tweet_text'].tolist()
        return process_batch(tweets, executor, model, schema, batch_size)




    with ThreadPoolExecutor(max_workers=max_workers) as pool:
        batches = [data.iloc[i:i + batch_size] for i in range(0, len(data), batch_size)]
        futures = [pool.submit(process_and_collect, batch) for batch in batches]
        for future in futures:
            results.append(future.result())




    return pd.DataFrame(results)




def save_results(data, filename):
    """Save data to JSON file."""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(f"Results saved in {filename}")
    except Exception as e:
        print(f"Error during saving: {e}")




# Assuming 'data_label_0' is your DataFrame filtered for label 0
processed_tweets = parallel_process_tweets(
    data=data_label_0,  # Use the data_label_0 DataFrame
    batch_size=10,      # Batch size
    max_workers=6,     # Number of parallel threads
    model=model,        # Your model variable
    schema=json_schema  # Your JSON schema variable
)




# Save the results to label_0_1.json
save_results(processed_tweets.to_dict(orient="records"), "label_0_1.json")

Results saved in label_0_1.json


In [17]:
import tiktoken
import pandas as pd
import json
import time
from concurrent.futures import ThreadPoolExecutor
from queue import Queue
import threading
import openai  # Make sure to import openai


def rate_limited_executor(max_calls_per_minute=100):
    """Rate-limited executor for API calls."""
    call_queue = Queue()


    def worker():
        while True:
            func, args, kwargs, result_queue = call_queue.get()
            try:
                result_queue.put(func(*args, **kwargs))
            except Exception as e:
                result_queue.put(e)
            call_queue.task_done()
            time.sleep(60 / max_calls_per_minute)


    for _ in range(max_calls_per_minute):
        threading.Thread(target=worker, daemon=True).start()


    def submit(func, *args, **kwargs):
        result_queue = Queue()
        call_queue.put((func, args, kwargs, result_queue))
        result = result_queue.get()
        if isinstance(result, Exception):
            raise result
        return result


    return submit




def process_batch(tweets, executor, model, schema, batch_size):
    """Process one batch of tweets."""
    messages = [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': json.dumps(tweets)}
    ]
    try:
        response = executor(
            client.chat.completions.create,
            model=model,
            messages=messages,
            temperature=0.7,
            response_format={"type": "json_object", "schema": schema},
            max_tokens=8192
        )
        return {'batch_size': len(tweets), 'response': response.choices[0].message.content}
    except Exception as e:
        return {'batch_size': len(tweets), 'error': str(e)}




def parallel_process_tweets(data, batch_size, max_workers, model, schema):
    """Parallel processing of tweets with rate-limiting."""
    executor = rate_limited_executor(max_calls_per_minute=60)
    results = []


    def process_and_collect(batch):
        tweets = batch['Tweet_text'].tolist()
        return process_batch(tweets, executor, model, schema, batch_size)


    with ThreadPoolExecutor(max_workers=max_workers) as pool:
        batches = [data.iloc[i:i + batch_size] for i in range(0, len(data), batch_size)]
        futures = [pool.submit(process_and_collect, batch) for batch in batches]
        for future in futures:
            results.append(future.result())


    return pd.DataFrame(results)




def save_results(data, filename):
    """Save data to JSON file."""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(f"Results saved in {filename}")
    except Exception as e:
        print(f"Error during saving: {e}")




# Assuming these are already defined in your previous code
# SYSTEM_PROMPT
# client (OpenAI client)
# model
# json_schema
# data_label_1 (DataFrame filtered for label 1)


# Process data_label_1 and save results
processed_tweets_label_1 = parallel_process_tweets(
    data=data_label_1,      # Use the data_label_1 DataFrame
    batch_size=10,          # Batch size
    max_workers=6,         # Number of parallel threads
    model=model,            # Your model variable
    schema=json_schema      # Your JSON schema variable
)


# Save the results to label_1_1.json
save_results(processed_tweets_label_1.to_dict(orient="records"), "label_1_1.json")

Results saved in label_1_1.json


In [18]:
import json
import networkx as nx
import plotly.graph_objects as go
import numpy as np


def load_json_file(file_path):
    """Load JSON file safely."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except Exception as e:
        print(f"Error loading JSON file {file_path}: {e}")
        return []


def create_theme_network(data_bot, data_nonbot, min_confidence=70):
    """Create a network graph from themes with a minimum confidence level."""
    G = nx.Graph()

    def process_data(data, is_bot=True):
        for item in data:
            response = item.get('response')
            if isinstance(response, str):
                try:
                    response_data = json.loads(response)
                    themes = response_data.get('overall_themes', [])

                    for theme in themes:
                        confidence = theme.get('confidence', 0)
                        if confidence >= min_confidence:
                            theme_name = theme['name']
                            keywords = theme.get('keywords', [])
                            sentiment_distribution = theme.get('sentiment_distribution', {})

                            # Add or update theme node
                            if not G.has_node(theme_name):
                                G.add_node(theme_name,
                                           type='theme',
                                           is_bot=is_bot,
                                           confidence=confidence,
                                           keywords=keywords,
                                           sentiment_distribution=sentiment_distribution)
                            else:
                                # If node exists and bot status is different, mark as mixed
                                current_bot_status = G.nodes[theme_name].get('is_bot')
                                if current_bot_status != is_bot:
                                    G.nodes[theme_name]['is_bot'] = 'mixed'

                except json.JSONDecodeError:
                    print(f"Invalid JSON in response: {response}")

    # Process bot and non-bot data
    process_data(data_bot, is_bot=True)
    process_data(data_nonbot, is_bot=False)

    return G


def visualize_network(G):
    """Visualize bot, non-bot, and mixed themes in clustered regions."""
    # Separate nodes by bot status
    bot_nodes = [node for node, data in G.nodes(data=True) if data.get('is_bot') == True]
    nonbot_nodes = [node for node, data in G.nodes(data=True) if data.get('is_bot') == False]
    mixed_nodes = [node for node, data in G.nodes(data=True) if data.get('is_bot') == 'mixed']

    # Create traces for each category
    traces = []

    # Function to determine node size based on confidence
    def get_node_size(confidence):
        return 5 + confidence / 10  # Adjust the scaling factor as needed

    # Bot nodes - clustered in bottom left
    if bot_nodes:
        bot_x = np.random.normal(loc=-5, scale=1, size=len(bot_nodes))
        bot_y = np.random.normal(loc=-5, scale=1, size=len(bot_nodes))
        bot_sizes = [get_node_size(G.nodes[node]['confidence']) for node in bot_nodes]
        bot_trace = go.Scatter(
            x=bot_x,
            y=bot_y,
            mode='markers',
            hovertext=[
                f"Theme: {node}<br>"
                f"Type: Bot<br>"
                f"Confidence: {G.nodes[node]['confidence']}%<br>"
                f"Keywords: {', '.join(G.nodes[node].get('keywords', []))}<br>"
                f"Sentiment Distribution: Positive {G.nodes[node].get('sentiment_distribution', {}).get('positive_percentage', 0)}%, "
                f"Negative {G.nodes[node].get('sentiment_distribution', {}).get('negative_percentage', 0)}%, "
                f"Neutral {G.nodes[node].get('sentiment_distribution', {}).get('neutral_percentage', 0)}%"
                for node in bot_nodes
            ],
            hoverinfo='text',
            marker=dict(
                size=bot_sizes,
                color='rgb(255,0,0)',  # Red
                line=dict(width=1, color='rgb(139,0,0)')  # Dark red
            ),
            name='Bot Themes'
        )
        traces.append(bot_trace)

    # Non-bot nodes - clustered in top right
    if nonbot_nodes:
        nonbot_x = np.random.normal(loc=5, scale=1, size=len(nonbot_nodes))
        nonbot_y = np.random.normal(loc=5, scale=1, size=len(nonbot_nodes))
        nonbot_sizes = [get_node_size(G.nodes[node]['confidence']) for node in nonbot_nodes]
        nonbot_trace = go.Scatter(
            x=nonbot_x,
            y=nonbot_y,
            mode='markers',
            hovertext=[
                f"Theme: {node}<br>"
                f"Type: Non-Bot<br>"
                f"Confidence: {G.nodes[node]['confidence']}%<br>"
                f"Keywords: {', '.join(G.nodes[node].get('keywords', []))}<br>"
                f"Sentiment Distribution: Positive {G.nodes[node].get('sentiment_distribution', {}).get('positive_percentage', 0)}%, "
                f"Negative {G.nodes[node].get('sentiment_distribution', {}).get('negative_percentage', 0)}%, "
                f"Neutral {G.nodes[node].get('sentiment_distribution', {}).get('neutral_percentage', 0)}%"
                for node in nonbot_nodes
            ],
            hoverinfo='text',
            marker=dict(
                size=nonbot_sizes,
                color='rgb(0,255,0)',  # Green
                line=dict(width=1, color='rgb(0,100,0)')  # Dark green
            ),
            name='Non-Bot Themes'
        )
        traces.append(nonbot_trace)

    # Mixed nodes - clustered in center
    if mixed_nodes:
        mixed_x = np.random.normal(loc=0, scale=1, size=len(mixed_nodes))
        mixed_y = np.random.normal(loc=0, scale=1, size=len(mixed_nodes))
        mixed_sizes = [get_node_size(G.nodes[node]['confidence']) for node in mixed_nodes]
        mixed_trace = go.Scatter(
            x=mixed_x,
            y=mixed_y,
            mode='markers',
            hovertext=[
                f"Theme: {node}<br>"
                f"Type: Mixed<br>"
                f"Confidence: {G.nodes[node]['confidence']}%<br>"
                f"Keywords: {', '.join(G.nodes[node].get('keywords', []))}<br>"
                f"Sentiment Distribution: Positive {G.nodes[node].get('sentiment_distribution', {}).get('positive_percentage', 0)}%, "
                f"Negative {G.nodes[node].get('sentiment_distribution', {}).get('negative_percentage', 0)}%, "
                f"Neutral {G.nodes[node].get('sentiment_distribution', {}).get('neutral_percentage', 0)}%"
                for node in mixed_nodes
            ],
            hoverinfo='text',
            marker=dict(
                size=mixed_sizes,
                color='rgb(128,0,128)',  # Purple
                line=dict(width=1, color='rgb(75,0,130)')  # Indigo
            ),
            name='Mixed Themes'
        )
        traces.append(mixed_trace)

    # Create figure
    fig = go.Figure(data=traces)

    # Update layout for better visualization
    fig.update_layout(
        title="Theme Network Visualization",
        showlegend=True,
        hovermode='closest',
        width=1000,
        height=800,
        xaxis=dict(
            title="X-axis",
            showgrid=False,
            zeroline=False,
            showticklabels=False
        ),
        yaxis=dict(
            title="Y-axis",
            showgrid=False,
            zeroline=False,
            showticklabels=False
        ),
        plot_bgcolor='rgb(240,240,240)',  # Light grey background
        legend=dict(
            x=0.85,
            y=1,
            bgcolor='rgba(255,255,255,0.5)',
            bordercolor='black',
            borderwidth=1
        )
    )

    fig.show()


# Example Usage
if __name__ == "__main__":
    # Load bot and non-bot data
    bot_data = load_json_file('label_1_1.json')  # Bot data
    nonbot_data = load_json_file('label_0_1.json')  # Non-bot data

    # Create and visualize the graph
    graph = create_theme_network(bot_data, nonbot_data, min_confidence=70)
    visualize_network(graph)


In [19]:
import json
import networkx as nx
import plotly.graph_objects as go
import numpy as np


def load_json_file(file_path):
    """Load JSON file safely."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except Exception as e:
        print(f"Error loading JSON file {file_path}: {e}")
        return []


def create_theme_network(data_bot, data_nonbot, min_confidence=70):
    """Create a network graph from themes with a minimum confidence level."""
    G = nx.Graph()

    def process_data(data, is_bot=True):
        for item in data:
            response = item.get('response')
            if isinstance(response, str):
                try:
                    response_data = json.loads(response)
                    themes = response_data.get('overall_themes', [])
                    for theme in themes:
                        confidence = theme.get('confidence', 0)
                        if confidence >= min_confidence:
                            theme_name = theme['name']
                            keywords = theme.get('keywords', [])
                            sentiment_distribution = theme.get('sentiment_distribution', {})
                            positive = sentiment_distribution.get('positive_percentage', 0)
                            negative = sentiment_distribution.get('negative_percentage', 0)
                            neutral = sentiment_distribution.get('neutral_percentage', 0)

                            if not G.has_node(theme_name):
                                G.add_node(theme_name,
                                           type='theme',
                                           is_bot=is_bot,
                                           confidence=confidence,
                                           keywords=keywords,
                                           sentiment_distribution={
                                               'positive': positive,
                                               'negative': negative,
                                               'neutral': neutral
                                           })
                            else:
                                current_bot_status = G.nodes[theme_name].get('is_bot')
                                if current_bot_status != is_bot:
                                    G.nodes[theme_name]['is_bot'] = 'mixed'
                except json.JSONDecodeError:
                    print(f"Invalid JSON in response: {response}")

    process_data(data_bot, is_bot=True)
    process_data(data_nonbot, is_bot=False)
    return G


def visualize_network(G):
    """Visualize network clustered by bot and nonbot themes with sentiment distribution."""
    if len(G.nodes) == 0:
        print("No nodes to visualize.")
        return

    bot_nodes = [node for node, data in G.nodes(data=True) if data['is_bot'] == True]
    nonbot_nodes = [node for node, data in G.nodes(data=True) if data['is_bot'] == False]
    mixed_nodes = [node for node, data in G.nodes(data=True) if data['is_bot'] == 'mixed']

    def get_node_size(confidence):
        return 5 + confidence / 10

    traces = []
    categories = [
        ('Bot Themes', bot_nodes, 'rgb(255,0,0)', -5),
        ('Non-Bot Themes', nonbot_nodes, 'rgb(0,255,0)', 5),
        ('Mixed Themes', mixed_nodes, 'rgb(128,0,128)', 0),
    ]

    for name, nodes, color, center in categories:
        if nodes:
            positions_x = np.random.normal(loc=center, scale=1, size=len(nodes))
            positions_y = np.random.normal(loc=center, scale=1, size=len(nodes))
            sizes = [get_node_size(G.nodes[node]['confidence']) for node in nodes]

            hover_texts = [
                f"Theme: {node}<br>"
                f"Type: {name}<br>"
                f"Confidence: {G.nodes[node]['confidence']}%<br>"
                f"Keywords: {', '.join(G.nodes[node].get('keywords', []))}<br>"
                f"Sentiments:<br>"
                f"Positive: {G.nodes[node]['sentiment_distribution']['positive']}%<br>"
                f"Negative: {G.nodes[node]['sentiment_distribution']['negative']}%<br>"
                f"Neutral: {G.nodes[node]['sentiment_distribution']['neutral']}%"
                for node in nodes
            ]

            traces.append(go.Scatter(
                x=positions_x,
                y=positions_y,
                mode='markers',
                marker=dict(
                    size=sizes,
                    color=color,
                    line=dict(width=1, color='black')
                ),
                hovertext=hover_texts,
                hoverinfo='text',
                name=name
            ))

    fig = go.Figure(data=traces)
    fig.update_layout(
        title="Theme Network Visualization (Clustered by Bot Status and Sentiments)",
        showlegend=True,
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        width=1200,
        height=800,
        plot_bgcolor='white',
        paper_bgcolor='white'
    )

    fig.show()


# Load JSON files
data_label_0 = load_json_file('label_0_1.json')  # Bot data
data_label_1 = load_json_file('label_1_1.json')  # Non-bot data

# Create network
network = create_theme_network(data_label_0, data_label_1, min_confidence=70)

# Visualize network
visualize_network(network)


In [20]:
import json
import networkx as nx
import plotly.graph_objects as go
import numpy as np


def load_json_file(file_path):
    """Load JSON file safely."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except Exception as e:
        print(f"Error loading JSON file {file_path}: {e}")
        return []


def create_theme_network(data_bot, data_nonbot, min_confidence=90):  # Changed default to 90
    """Create a network graph from themes with a minimum confidence level."""
    G = nx.Graph()


    def process_data(data, is_bot=True):
        for item in data:
            response = item.get('response')
            if isinstance(response, str):
                try:
                    response_data = json.loads(response)
                    themes = response_data.get('overall_themes', [])
                    for theme in themes:
                        confidence = theme.get('confidence', 0)
                        if confidence >= min_confidence:  # Only process themes with 90%+ confidence
                            theme_name = theme['name']
                            keywords = theme.get('keywords', [])
                            sentiment_distribution = theme.get('sentiment_distribution', {})
                            positive = sentiment_distribution.get('positive_percentage', 0)
                            negative = sentiment_distribution.get('negative_percentage', 0)
                            neutral = sentiment_distribution.get('neutral_percentage', 0)
                            
                            if not G.has_node(theme_name):
                                G.add_node(theme_name,
                                           type='theme',
                                           is_bot=is_bot,
                                           confidence=confidence,
                                           keywords=keywords,
                                           sentiment_distribution={
                                               'positive': positive,
                                               'negative': negative,
                                               'neutral': neutral
                                           })
                            else:
                                current_bot_status = G.nodes[theme_name].get('is_bot')
                                if current_bot_status != is_bot:
                                    G.nodes[theme_name]['is_bot'] = 'mixed'
                except json.JSONDecodeError:
                    print(f"Invalid JSON in response: {response}")


    process_data(data_bot, is_bot=True)
    process_data(data_nonbot, is_bot=False)
    return G


def visualize_network(G):
    """Visualize network clustered by bot and nonbot themes with sentiment distribution."""
    if len(G.nodes) == 0:
        print("No nodes to visualize.")
        return


    bot_nodes = [node for node, data in G.nodes(data=True) if data['is_bot'] == True]
    nonbot_nodes = [node for node, data in G.nodes(data=True) if data['is_bot'] == False]
    mixed_nodes = [node for node, data in G.nodes(data=True) if data['is_bot'] == 'mixed']


    def get_node_size(confidence):
        return 5 + confidence / 10


    traces = []
    categories = [
        ('Bot Themes', bot_nodes, 'rgb(255,0,0)', -5),
        ('Non-Bot Themes', nonbot_nodes, 'rgb(0,255,0)', 5),
        ('Mixed Themes', mixed_nodes, 'rgb(128,0,128)', 0),
    ]


    for name, nodes, color, center in categories:
        if nodes:
            positions_x = np.random.normal(loc=center, scale=1, size=len(nodes))
            positions_y = np.random.normal(loc=center, scale=1, size=len(nodes))
            sizes = [get_node_size(G.nodes[node]['confidence']) for node in nodes]


            hover_texts = [
                f"Theme: {node}<br>"
                f"Type: {name}<br>"
                f"Confidence: {G.nodes[node]['confidence']}%<br>"
                f"Keywords: {', '.join(G.nodes[node].get('keywords', []))}<br>"
                f"Sentiments:<br>"
                f"Positive: {G.nodes[node]['sentiment_distribution']['positive']}%<br>"
                f"Negative: {G.nodes[node]['sentiment_distribution']['negative']}%<br>"
                f"Neutral: {G.nodes[node]['sentiment_distribution']['neutral']}%"
                for node in nodes
            ]


            traces.append(go.Scatter(
                x=positions_x,
                y=positions_y,
                mode='markers',
                marker=dict(
                    size=sizes,
                    color=color,
                    line=dict(width=1, color='black')
                ),
                hovertext=hover_texts,
                hoverinfo='text',
                name=name
            ))


    fig = go.Figure(data=traces)
    fig.update_layout(
        title="Theme Network Visualization (90%+ Confidence, Clustered by Bot Status and Sentiments)",
        showlegend=True,
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        width=1200,
        height=800,
        plot_bgcolor='white',
        paper_bgcolor='white'
    )


    # Print some statistics
    print(f"Total Themes (90%+ Confidence): {len(G.nodes())}")
    print(f"Bot Themes: {len(bot_nodes)}")
    print(f"Non-Bot Themes: {len(nonbot_nodes)}")
    print(f"Mixed Themes: {len(mixed_nodes)}")


    fig.show()


# Load JSON files
data_label_0 = load_json_file('label_0_1.json')  # Bot data
data_label_1 = load_json_file('label_1_1.json')  # Non-bot data


# Create network with 90% minimum confidence
network = create_theme_network(data_label_0, data_label_1, min_confidence=90)


# Visualize network
visualize_network(network)

Total Themes (90%+ Confidence): 11
Bot Themes: 8
Non-Bot Themes: 2
Mixed Themes: 1
